In [1]:
!pip install transformers 
!pip install datasets 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 8.3 MB/s 
     |████████████████████████████████| 6.6 MB 48.4 MB/s 
     |████████████████████████████████| 596 kB 65.5 MB/s 
     |████████████████████████████████| 101 kB 14.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 7.6 MB/s 
     |████████████████████████████████| 141 kB 67.3 MB/s 
     |████████████████████████████████| 212 kB 61.8 MB/s 
     |████████████████████████████████| 115 kB 67.6 MB/s 
     |████████████████████████████████| 127 kB 68.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully u

In [2]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import torch
import transformers
torch.__version__, transformers.__version__

('1.12.0+cu113', '4.21.1')

In [3]:
torch.cuda.is_available() 

True

In [4]:
df = pd.read_csv('sample.csv')
df.head(10) 

,Industry,CompName,year,filename,sentence_id,team,day,sentence,Label
0,Energy,CHEVRON CORP,2002,CHEVRON CORP_2002.txt,8,1,1,Product Stewardship Manage potential risks of ...,PL
1,Energy,CHEVRON CORP,2002,CHEVRON CORP_2002.txt,10,1,1,CNL and UNDP will develop a coordinated master...,CR
2,Energy,CHEVRON CORP,2003,CHEVRON CORP_2003.txt,30,1,1,ChevronTexaco continues to advocate for consis...,CC
3,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,38,1,1,"Maturing oil fields, more technically challeng...",NC
4,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,41,1,1,We found that units in which employees rated t...,HC
5,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,48,1,1,Payments to international officials or parties...,BE
6,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,53,1,1,Chevron Energy Solutions In addition to focusi...,CC
7,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,55,1,1,We also have a 24-hour-a-day global hotline th...,PL
8,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,56,1,1,"In 2004, we spilled 15,514 barrels of oil, whi...",PW
9,Energy,CHEVRON CORP,2004,CHEVRON CORP_2004.txt,58,1,1,"Although not required by regulations, the loca...",NC


In [5]:
df = df.drop(['Industry', 'year', 'CompName', 'filename', 'sentence_id', 
              'team', 'day'], axis=1)
df.head(10)

,sentence,Label
0,Product Stewardship Manage potential risks of ...,PL
1,CNL and UNDP will develop a coordinated master...,CR
2,ChevronTexaco continues to advocate for consis...,CC
3,"Maturing oil fields, more technically challeng...",NC
4,We found that units in which employees rated t...,HC
5,Payments to international officials or parties...,BE
6,Chevron Energy Solutions In addition to focusi...,CC
7,We also have a 24-hour-a-day global hotline th...,PL
8,"In 2004, we spilled 15,514 barrels of oil, whi...",PW
9,"Although not required by regulations, the loca...",NC


In [6]:
df.isnull().any()

sentence    False
Label       False
dtype: bool

In [7]:
df.shape

(4500, 2)

In [8]:
from sklearn.preprocessing import LabelEncoder

In [9]:
encoder = LabelEncoder() 
df['Label'] = encoder.fit_transform(df['Label'])

In [10]:
df.head()

,sentence,Label
0,Product Stewardship Manage potential risks of ...,7
1,CNL and UNDP will develop a coordinated master...,3
2,ChevronTexaco continues to advocate for consis...,1
3,"Maturing oil fields, more technically challeng...",6
4,We found that units in which employees rated t...,4


In [11]:
df['Label'].value_counts() 

7    500
3    500
1    500
6    500
4    500
0    500
8    500
2    500
5    500
Name: Label, dtype: int64

In [12]:
df = df.rename(columns={"Label": "label"})

In [13]:
df.head()

,sentence,label
0,Product Stewardship Manage potential risks of ...,7
1,CNL and UNDP will develop a coordinated master...,3
2,ChevronTexaco continues to advocate for consis...,1
3,"Maturing oil fields, more technically challeng...",6
4,We found that units in which employees rated t...,4


In [14]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(3645, 2) (450, 2) (405, 2)


In [15]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=9)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Some weights of the model checkpoint at yiyanghkust/finbert-pretrain were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [16]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
dataset_train

Dataset({
    features: ['sentence', 'Label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 3645
})

In [17]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions, labels)}

In [18]:
args = TrainingArguments(
        output_dir = 'temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         
        args=args,                 
        train_dataset=dataset_train,         
        eval_dataset=dataset_val,           
        compute_metrics=compute_metrics
)

trainer.train()   

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3645
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 570


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.512448,0.859259
2,No log,0.428932,0.859259
3,No log,0.410860,0.879012
4,No log,0.418548,0.883951
5,0.460600,0.426315,0.883951


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 405
  Batch size = 32
Saving model checkpoint to temp/checkpoint-114
Configuration saved in temp/checkpoint-114/config.json
Model weights saved in temp/checkpoint-114/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 405
  Batch size = 32
Saving model checkpoint to temp/checkpoint-228
Configuration saved in temp/checkpoint-228

TrainOutput(global_step=570, training_loss=0.4155931221811395, metrics={'train_runtime': 441.1362, 'train_samples_per_second': 41.314, 'train_steps_per_second': 1.292, 'total_flos': 1198875090758400.0, 'train_loss': 0.4155931221811395, 'epoch': 5.0})

In [19]:
model.eval()
trainer.predict(dataset_test).metrics

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 450
  Batch size = 32


{'test_accuracy': 0.8711111111111111,
 'test_loss': 0.4993882477283478,
 'test_runtime': 3.9247,
 'test_samples_per_second': 114.657,
 'test_steps_per_second': 3.822}

In [20]:
trainer.save_model('finbert-sentiment/')

Saving model checkpoint to finbert-sentiment/
Configuration saved in finbert-sentiment/config.json
Model weights saved in finbert-sentiment/pytorch_model.bin
